In [1]:
import numpy as np
import imgaug as ia
import imgaug.augmenters as iaa
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
import operator
import tensorflow as tf
import random
from keras.preprocessing.image import ImageDataGenerator
import skimage
from skimage.io import imread, imshow, imsave

In [2]:
def crop_center(img, bounding):
    start = tuple(map(lambda a, da: a//2-da//2, img.shape, bounding))
    end = tuple(map(operator.add, start, bounding))
    slices = tuple(map(slice, start, end))
    return img[slices]

In [3]:
'''Source Path'''
train_all_path = r'F:\Leuk study re-designed\C-NMC\High imbalance\Train - 1 to 102 ratio\all'
train_hem_path = r'F:\Leuk study re-designed\C-NMC\High imbalance\Train - 1 to 102 ratio\hem'  # augment this

# val_all_path = r'D:\new_leuk\Leukemia_Work_Revive\Corrected\Originall data\val\all'
# val_hem_path = r'D:\new_leuk\Leukemia_Work_Revive\Corrected\Originall data\val\hem'     # augment this

In [ ]:
train_all_list = os.listdir(train_all_path)
train_all_list.sort()

train_hem_list = os.listdir(train_hem_path)       # augment this
train_hem_list.sort()

# val_all_list = os.listdir(val_all_path)
# val_all_list.sort()

# val_hem_list = os.listdir(val_hem_path)          # augment this
# val_hem_list.sort()

print('train/all: ', len(train_all_list))
print('train/hem :', len(train_hem_list))
# print('val/all: ', len(val_all_list))
# print('val/hem :', len(val_hem_list))

In [ ]:
train_all_batch = np.zeros((len(train_all_list), 210, 210, 3), dtype=np.uint8)
train_hem_batch = np.zeros((len(train_hem_list), 210, 210, 3), dtype=np.uint8)    # augment this

# val_all_batch = np.zeros((len(val_all_list), 210, 210, 3), dtype=np.uint8)
# val_hem_batch = np.zeros((len(val_hem_list), 210, 210, 3), dtype=np.uint8)        # augment this

print(train_all_batch.shape, train_hem_batch.shape)
#print(val_all_batch.shape, val_hem_batch.shape)

In [6]:
def Read_n_Crop(list_data, batch, path):
    i=0
    for x in list_data:
        image = imread(os.path.join(path, x))
        image = crop_center(image, (210,210,3))
        batch[i] = image
        i+=1
    
    print(type(batch), batch.shape, batch.dtype, batch[0].shape, batch[0].dtype)

In [ ]:
Read_n_Crop(train_all_list, train_all_batch, train_all_path)
Read_n_Crop(train_hem_list, train_hem_batch, train_hem_path)      # augment this

# Read_n_Crop(val_all_list, val_all_batch, val_all_path)
# Read_n_Crop(val_hem_list, val_hem_batch, val_hem_path)           # augment this

In [8]:
'''Save Oversampled Cropped Images'''
def SaveOC_images(list_data, path, batch):
    for x in range(len(list_data)):
#         print(path)
        new_path = path + '/' + list_data[x]
        print(new_path)
#         break
        imsave(new_path ,batch[x])

In [ ]:
path = 'F:/Leuk study re-designed/C-NMC/High imbalance/10x Aug/all'
SaveOC_images(train_all_list, path, train_all_batch)
path = 'F:/Leuk study re-designed/C-NMC/High imbalance/10x Aug/hem'
SaveOC_images(train_hem_list, path, train_hem_batch)     # augment this

# path = 'D:/new_leuk/Leukemia_Work_Revive/Corrected/Aug_5x/val/all'
# SaveOC_images(val_all_list, path, val_all_batch)
# path = 'D:/new_leuk/Leukemia_Work_Revive/Corrected/Aug_5x/val/hem'
# SaveOC_images(val_hem_list, path, val_hem_batch)         # augment this

In [10]:
def Plot_images(batch, num):
    i=0
    plt.figure(figsize=(15,15))
    for x in batch:
        if 0<=i<(num*num):
            plt.subplot(num,num,i+1)
            plt.imshow(x)
        else:
            break
        i +=1

In [ ]:
num = 8 # row n col
Plot_images(train_hem_batch, num)

In [12]:
# num = 5
# Plot_images(val_hem_batch, num)

In [13]:
# '''Define Augmentation Pipeline'''

# sometimes = lambda aug: iaa.Sometimes(0.5, aug)
# seq = iaa.Sequential(
#     [
#         # apply the following augmenters to most images
#         iaa.Fliplr(0.5), # horizontally flip 50% of all images
#         iaa.Flipud(0.2), # vertically flip 20% of all images
#         # crop images by -5% to 10% of their height/width
#         sometimes(iaa.CropAndPad(
#             percent=(-0.05, 0.1),
#             pad_mode=ia.ALL,
#             pad_cval=(0, 255)
#         )),
#         sometimes(iaa.Affine(
#             scale={"x": (0.8, 1.2), "y": (0.8, 1.2)}, # scale images to 80-120% of their size, individually per axis
#             translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)}, # translate by -20 to +20 percent (per axis)
#             rotate=(-45, 45), # rotate by -45 to +45 degrees
#             shear=(-16, 16), # shear by -16 to +16 degrees
#             order=[0, 1], # use nearest neighbour or bilinear interpolation (fast)
#             cval=(0, 255), # if mode is constant, use a cval between 0 and 255
#             mode=ia.ALL # use any of scikit-image's warping modes (see 2nd image from the top for examples)
#         )),
#         #'''
#         # execute 0 to 5 of the following (less important) augmenters per image
#         # don't execute all of them, as that would often be way too strong
#         iaa.SomeOf((0, 5),
#             [
#                 sometimes(iaa.Superpixels(p_replace=(0, 1.0), n_segments=(20, 200))), # convert images into their superpixel representation
#                 iaa.OneOf([
#                     iaa.GaussianBlur((0, 3.0)), # blur images with a sigma between 0 and 3.0
#                     iaa.AverageBlur(k=(2, 7)), # blur image using local means with kernel sizes between 2 and 7
#                     iaa.MedianBlur(k=(3, 11)), # blur image using local medians with kernel sizes between 2 and 7
#                 ]),
#                 iaa.Sharpen(alpha=(0, 1.0), lightness=(0.75, 1.5)), # sharpen images
#                 iaa.Emboss(alpha=(0, 1.0), strength=(0, 2.0)), # emboss images
#                 # search either for all edges or for directed edges,
#                 # blend the result with the original image using a blobby mask
#                 iaa.BlendAlphaSimplexNoise(iaa.OneOf([
#                     iaa.EdgeDetect(alpha=(0.5, 1.0)),
#                     iaa.DirectedEdgeDetect(alpha=(0.5, 1.0), direction=(0.0, 1.0)),
#                 ])),
#                 iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5), # add gaussian noise to images
#                 iaa.OneOf([
#                     iaa.Dropout((0.01, 0.1), per_channel=0.5), # randomly remove up to 10% of the pixels
#                     iaa.CoarseDropout((0.03, 0.15), size_percent=(0.02, 0.05), per_channel=0.2),
#                 ]),
#                 iaa.Invert(0.05, per_channel=True), # invert color channels
#                 iaa.Add((-10, 10), per_channel=0.5), # change brightness of images (by -10 to 10 of original value)
#                 iaa.AddToHueAndSaturation((-20, 20)), # change hue and saturation
#                 # either change the brightness of the whole image (sometimes
#                 # per channel) or change the brightness of subareas
#                 iaa.OneOf([
#                     iaa.Multiply((0.5, 1.5), per_channel=0.5),
#                     iaa.BlendAlphaFrequencyNoise(
#                         exponent=(-4, 0),
#                         foreground=iaa.Multiply((0.5, 1.5), per_channel=True),
#                         background=iaa.LinearContrast((0.5, 2.0))
#                     )
#                 ]),
#                 iaa.LinearContrast((0.5, 2.0), per_channel=0.5), # improve or worsen the contrast
#                 iaa.Grayscale(alpha=(0.0, 1.0)),
#                 sometimes(iaa.ElasticTransformation(alpha=(0.5, 3.5), sigma=0.25)), # move pixels locally around (with random strengths)
#                 sometimes(iaa.PiecewiseAffine(scale=(0.01, 0.05))), # sometimes move parts of the image around
#                 sometimes(iaa.PerspectiveTransform(scale=(0.01, 0.1)))
#             ],
#             #'''
#             random_order=True
#         )
#     ],
#     random_order=True
# )


In [14]:
sometimes = lambda aug: iaa.Sometimes(0.5, aug)
seq = iaa.Sequential([
    # apply the following augmenters to most images
        iaa.Fliplr(0.5), # horizontally flip 50% of all images
        iaa.Flipud(0.2), # vertically flip 20% of all images
        # crop images by -5% to 10% of their height/width
         iaa.Crop(percent=(0, 0.1)), # random crops
    
        sometimes(iaa.CropAndPad(
            pad_mode=ia.ALL,          
            percent=(-0.05, 0.1),
            pad_cval=(0, 255)
        )),
    
      sometimes(iaa.Affine(
            scale={"x": (0.8, 1.2), "y": (0.8, 1.2)}, # scale images to 80-120% of their size, individually per axis
            translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)}, # translate by -20 to +20 percent (per axis)
            rotate=(-45, 45), # rotate by -45 to +45 degrees
            #shear=(-16, 16), # shear by -16 to +16 degrees
            #order=[0, 1], # use nearest neighbour or bilinear interpolation (fast)
            #cval=(0, 255), # if mode is constant, use a cval between 0 and 255
            #mode=ia.ALL # use any of scikit-image's warping modes (see 2nd image from the top for examples)
        )),
],
random_order=True
)

In [15]:
def AugmentNsave_images(seq, batch, list_data, path, n):
    '''Augment'''
    images_aug = seq(images=batch)
    print(type(images_aug), images_aug.shape, images_aug.dtype, np.max(images_aug), np.min(images_aug), images_aug[0].shape, images_aug[0].dtype, np.max(images_aug[0]), np.min(images_aug[0]))
    '''Save'''
    for x in range(len(list_data)):
        #print(path)
        new_path = path + '/' + str(n) + 'Aug_' + list_data[x]
        print(new_path)
        imsave(new_path ,images_aug[x])

In [ ]:
'''Each time augmented results are different'''
times2augment = 10
path = 'F:/Leuk study re-designed/C-NMC/High imbalance/10x Aug/all'
for x in range(times2augment):
    AugmentNsave_images(seq,train_all_batch, train_all_list, path, x+1)
    
# path = 'D:/new_leuk/Leukemia_Work_Revive/Aug_10x/val/hem'
# for x in range(times2augment):
#     AugmentNsave_images(seq,val_hem_batch, val_hem_list, path, x+1)